In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer

https://kavita-ganesan.com/tfidftransformer-tfidfvectorizer-usage-differences/#.Y5R1cOzMLVY




In natural language processing (NLP), the term frequency-inverse document frequency (TF-IDF) score is a measure of the importance of a word or phrase to the meaning of a document (or group of documents) in a collection. The TF-IDF score is calculated by combining the term frequency (TF) and the inverse document frequency (IDF) of the word or phrase.

The term frequency (TF) of a word or phrase is the number of times it appears in a document. This value reflects how important the word is to the meaning of the document.

The inverse document frequency (IDF) of a word or phrase is a measure of how frequently it appears in the entire collection of documents. This value reflects how common the word is across the collection of documents. Words that appear frequently in the collection are given a lower IDF score, while words that appear infrequently are given a higher IDF score.

The TF-IDF score is calculated by multiplying the TF and IDF values for a word or phrase. This score reflects both the importance of the word to the meaning of the document and its rarity across the collection of documents. Words and phrases with high TF-IDF scores are considered to be more important and relevant to the meaning of the document than those with low TF-IDF scores.

In summary, the difference between the IDF and the TF-IDF score is that the IDF is a measure of the rarity of a word or phrase across a collection of documents, while the TF-IDF score is a measure of the importance of the word or phrase to the meaning of a specific document in the collection. The TF-IDF score combines both of these factors to give a more complete picture of the significance of a word or phrase to the meaning of a document.

In [2]:
# Load the text data into a DataFrame
data = pd.read_csv('data/p_content.csv')


# settings that you use for count vectorizer will go here 
tfidf_vectorizer=TfidfVectorizer(use_idf=True) 
# just send in all your docs here 
tfidf_vectorizer_vectors=tfidf_vectorizer.fit_transform(data['content'])



# get the first vector out (for the first document) 
first_vector_tfidfvectorizer=tfidf_vectorizer_vectors[0] 
# place tf-idf values in a pandas data frame 
df = pd.DataFrame(first_vector_tfidfvectorizer.T.todense(), index=tfidf_vectorizer.get_feature_names(), columns=["tfidf"]) 
df.sort_values(by=["tfidf"],ascending=False)

/opt/homebrew/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,tfidf
stoff,0.415353
kategorie,0.254226
schützen,0.220871
medizinisch,0.140170
partikelfiltrierender,0.138451
...,...
frühzeitig,0.000000
frühmorgen,0.000000
frühjahr,0.000000
frühestens,0.000000


The lower the IDF value of a word, the less unique it is to any particular document

In [3]:
#instantiate CountVectorizer() 
cv=CountVectorizer() 
# this steps generates word counts for the words in your docs 
word_count_vector=cv.fit_transform(data['content'])
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True) 
tfidf_transformer.fit(word_count_vector)

# print idf values 
df_idf = pd.DataFrame(tfidf_transformer.idf_, index=cv.get_feature_names_out(),columns=["idf_weights"]) 
# sort ascending 
df_idf.sort_values(by=['idf_weights'])


,idf_weights
der,1.000000
foto,1.069680
maskenpflicht,1.080043
werden,1.201740
in,1.213574
...,...
harmlos,4.951244
dreistufenplan,4.951244
oberbürgermeister,4.951244
obergrenze,4.951244


build feature frame

In [4]:
# Convert the matrix of TF-IDF values to a DataFrame
tfidf_df = pd.DataFrame(tfidf_vectorizer_vectors.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

newcol = ['tfidf_'+n for n in tfidf_df.columns]
#multicol1 = pd.MultiIndex.from_tuples(newcol)

tfidf_df.columns = newcol

tfidf_df.head()

,tfidf_0800,tfidf_10,tfidf_100,tfidf_102,tfidf_1051831326hundewiese,tfidf_1056853079maske,tfidf_108,tfidf_10m2,tfidf_11,tfidf_116,...,tfidf_übertragung,tfidf_übertrieben,tfidf_überwiegen,tfidf_überwinden,tfidf_überzeugen,tfidf_üblich,tfidf_übrig,tfidf_übrigens,tfidf_übriges,tfidf_übung
0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,0.046990,0.000000,0.0,0.0,0.0,0.0,0.071076,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,0.046508,0.051715,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.053589,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.065759


In [5]:
df_merged = data[['ID_GodotObject']].merge(tfidf_df, left_index=True, right_index=True)
df_merged.head()

,ID_GodotObject,tfidf_0800,tfidf_10,tfidf_100,tfidf_102,tfidf_1051831326hundewiese,tfidf_1056853079maske,tfidf_108,tfidf_10m2,tfidf_11,...,tfidf_übertragung,tfidf_übertrieben,tfidf_überwiegen,tfidf_überwinden,tfidf_überzeugen,tfidf_üblich,tfidf_übrig,tfidf_übrigens,tfidf_übriges,tfidf_übung
0,2000115059032,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,2000116305030,0.046990,0.000000,0.0,0.0,0.0,0.0,0.071076,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,2000116325081,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,2000116346340,0.046508,0.051715,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,2000116371728,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.053589,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.065759


https://www.kaggle.com/code/pierremegret/gensim-word2vec-tutorial/notebook

In [6]:
df_merged.to_csv('data/feature/tfidf_content.csv', encoding='utf-8', index=False)